In [88]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
from urllib2 import urlopen # python2
import urllib2
import time

In [2]:
# SIDER에서 제공하는 indication 정보
df_sider_indication_raw = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/meddra_all_indications_header.tsv")
df_sider_indication_pt = df_sider_indication_raw[df_sider_indication_raw.MedDRA_concept_type == 'PT']
df_sider_indication = df_sider_indication_pt[['STITCH_compound_id_flat', 'UMLS_concept_id_for_MedDRA','MedDRA_concept_name']].drop_duplicates()

In [28]:
# SIDER에서 제공하는 indication 정보를 DrugBank_ID와 매칭하기 위해 STITCH id가 필요함.
df_stitch = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv")

In [57]:
print len(df_sider_indication) # sider의 indication 총 개수
print len(df_sider_indication['STITCH_compound_id_flat'].drop_duplicates()) # indication 정보를 가지고 있는 compound의 개수-->1437개

# stitch_pc id mapping 파일에서 sider에 indication 정보가 존재하는 stitch id를 솎아낸다.
df_query = df_stitch[df_stitch['flat_chemical'].isin(df_sider_indication['STITCH_compound_id_flat'])] 
print len(df_query) # stitch id와 mapping되는 pc의 개수(pc는 중복 없음)
print len(df_query.drop_duplicates('flat_chemical')) #stitch_id의 개수 

15083
1437
45598
1437


In [ ]:
df_query.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_0_query_pubchemID.tsv", sep='\t', index=False)

In [ ]:
df_query = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_0_query_pubchemID.tsv")

In [77]:
print len(df_query)
print len(df_query.drop_duplicates('flat_chemical'))
df_query.head(1)

45598
1437


,flat_chemical,stereo_chemical,source_name,source_id
136,CID100002735,CID010000117,PC,10000117


In [87]:
# using UniChem REST API, convert pubchemID to DrugBank ID
results = pd.DataFrame()
  
start = 0
end = 11400
cnt = start
url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
print'%d : ' % cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_query[start:end].iterrows():
    record = row.tolist()
    stitch_id_flat = record[0]
    stitch_id_stereo = record[1]
    pubchem_id = record[3]
    url = url_base + str(pubchem_id) + '/22/2'
        
    MAX_ATTEMPTS = 8
    for attempt in range(MAX_ATTEMPTS):
        try:
            f = urlopen(url)
        except urllib2.HTTPError, e:
            print row
            if e.code == 404:
                cnt = cnt + 1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                break
            else:
                print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                break
                #raise
        except urllib2.URLError, e:
            if e.args[0][0] == 110: # connection time out
                sleep_secs = attempt ** 2
                print e , url
                print '  cnt = %d, error time : ' % (cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print '    Retrying in %d seconds...' % sleep_secs
                time.sleep(sleep_secs)
                continue 
            else:
                print row
                print e
                print '   cnt : %d, error time : ' % (cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                cnt = cnt +1
                break
        else: # error가 일어나지 않을 때          
            all_data = f.readlines()
            if all_data[0] == '[]':
                cnt = cnt + 1
            else:
                db_id=all_data[0] 
                temp_df = pd.DataFrame(data = [[stitch_id_flat, stitch_id_stereo, pubchem_id, db_id]], 
                                       columns = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID'])  
                results = results.append(temp_df)
                cnt = cnt + 1
            if cnt % 1000 == 0 :
                print '%d : '% cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            break
    else:#connection을 8번 시도했는데 안될 경우
        print 'we failed to reconnect 8 times in %s' % url
            
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
results.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_1_raw_SIDER_indication_DrugBank_id_0_11399.tsv", sep='\t', index=False)


0 : 2017-03-26 20:35:44
1000 : 2017-03-26 20:53:39
2000 : 2017-03-26 21:11:34
flat_chemical      CID100003780
stereo_chemical    CID013019157
source_name                  PC
source_id              13019157
Name: 2486402, dtype: object
3000 : 2017-03-26 21:29:29
flat_chemical      CID100002171
stereo_chemical    CID013083120
source_name                  PC
source_id              13083120
Name: 2508311, dtype: object
flat_chemical      CID116129672
stereo_chemical    CID016129672
source_name                  PC
source_id              16129672
Name: 3460788, dtype: object
flat_chemical      CID116129703
stereo_chemical    CID016129703
source_name                  PC
source_id              16129703
Name: 3460815, dtype: object
flat_chemical      CID116130199
stereo_chemical    CID016130199
source_name                  PC
source_id              16130199
Name: 3461273, dtype: object
flat_chemical      CID116132265
stereo_chemical    CID016132265
source_name                  PC
source_id     

In [90]:
results2 = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_1_raw_SIDER_indication_DrugBank_id_11400_22799.tsv")
results3 = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_1_raw_SIDER_indication_DrugBank_id_22800_34199.tsv")
results4 = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_1_raw_SIDER_indication_DrugBank_id_45600_45598.tsv")

In [91]:
frames = [results, results2, results3, results4]
concatenated = pd.concat(frames)

In [93]:
print len(concatenated)
print len(concatenated.drop_duplicates())

1293
1293


In [94]:
# SIDER에서 제공하는 indication 정보를 가지고 있는 DrugBank ID(raw 파일)
concatenated.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/independent_data2/11_1_raw_SIDER_indication_DrugBank_id_all.tsv", sep='\t', index=False)
concatenated.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/independent_data2/11_1_raw_SIDER_indication_DrugBank_id_all.tsv", sep='\t', index=False)

In [96]:
concatenated.head(2)

,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID100001003,CID000001003,1003,"[{""src_compound_id"":""DB02831""}]"
0,CID110028615,CID010077129,10077130,"[{""src_compound_id"":""DB09030""}]"


In [97]:
tidy = pd.DataFrame()
for index, row in concatenated.iterrows():
    record = row.tolist()
    db_ids = record[3].split(',')
    for i in db_ids:
        db_id = i.split(':')[1].lstrip('\"').rstrip('\"}]')
        temp_df = pd.DataFrame([[record[0], record[1], record[2], db_id]], columns = ['STITCH_compound_id_flat',
                                                                                      'STITCH_compound_id_stereo', 
                                                                                      'PubChem_ID', 'DrugBank_ID'])
        tidy = tidy.append(temp_df)
tidy.index = range(0, len(tidy))

In [100]:
print len(tidy)
tidy.head(2)

1318


,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID100001003,CID000001003,1003,DB02831
1,CID110028615,CID010077129,10077130,DB09030


In [101]:
# SIDER에서 제공하는 indication 정보를 가지고 있는 DrugBank ID(raw 파일)
tidy.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/independent_data2/11_2_SIDER_indication_DrugBank_id_tidy.tsv", sep='\t', index=False)
tidy.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/independent_data2/11_2_SIDER_indication_DrugBank_id_tidy.tsv", sep='\t', index=False)

In [151]:
# PubChem ID 빼고 중복제거 한것.
except_pc = tidy[['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'DrugBank_ID']].drop_duplicates()
print len(tidy)
print len(except_pc)
except_pc.head(1)

1318
1216


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID
0,CID100001003,CID000001003,DB02831


In [199]:
ind_umls_db = pd.merge(right = df_sider_indication, left =except_pc, right_on = 'STITCH_compound_id_flat', left_on='STITCH_compound_id_flat')
ind_umls_db2 = ind_umls_db[['DrugBank_ID', 'UMLS_concept_id_for_MedDRA']].drop_duplicates()
ind_umls_db3 = ind_umls_db[['DrugBank_ID', 'UMLS_concept_id_for_MedDRA', 'MedDRA_concept_name']].drop_duplicates()
print len(ind_umls_db)
print len(ind_umls_db2)
print len(ind_umls_db3)
ind_umls_db3.head()

13979
13704
13704


,DrugBank_ID,UMLS_concept_id_for_MedDRA,MedDRA_concept_name
0,DB02831,C0085682,Hypophosphataemia
2,DB09030,C0003850,Arteriosclerosis
3,DB09030,C0027051,Myocardial infarction
4,DB09030,C0948089,Acute coronary syndrome
5,DB09030,C0038454,Cerebrovascular accident


In [201]:
ind_umls_db.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/2_DrugBank_id_SIDER_id_indication.tsv", sep='\t', index=False)
ind_umls_db.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/2_DrugBank_id_SIDER_id_indication.tsv", sep='\t', index=False)

In [202]:
ind_umls_db3.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/2_2_DrugBank_id_SIDER_indication.tsv", sep='\t', index=False)
ind_umls_db3.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/2_2_DrugBank_id_SIDER_indication.tsv", sep='\t', index=False)

In [224]:
# gold association이 아닌 약의 indication 정보를 추출한다!
gold = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/2_gold_asso.tsv")
print len(gold)
gold.head(1)

1362


,DrugBank_ID,OMIM_ID
0,DB00659,103780


In [206]:
omim_umls = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/1_omim_umls_mapping_association.tsv")
print len(omim_umls)
omim_umls.head(1)

608


,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name
0,102100,"Acromegaloid Changes, Cutis Verticis Gyrata, A...",C1868756,Corneal leukoma


In [210]:
gold_asso_umls = pd.merge(right = gold, left = omim_umls) # gold asso의 disease(omim id)를 umls id로 mapping
print len(gold_asso_umls)
gold_asso_umls.head(2)

3478


,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,DrugBank_ID
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,DB00564
1,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,DB00190


In [211]:
temp = gold_asso_umls[['DrugBank_ID', 'OMIM_ID', 'UMLS_concept_ID']] # gold association을 drugbank_id , omim_id, umls id로 동시에 표현한것.
print len(temp)
print len(temp.drop_duplicates())
print len(temp.drop_duplicates('DrugBank_ID'))
print len(temp.drop_duplicates('OMIM_ID'))
print len(temp.drop_duplicates('UMLS_concept_ID')) #--> 수가 위보다 크다는건, omim id 한개당 여러개의 umls id가 matching되는 경우도 있다는 뜻.

3478
3347
446
235
376


In [212]:
gold_umls = temp['UMLS_concept_ID'].drop_duplicates() #gold disease를 umls로 표현했을 때
gold_drug = temp['DrugBank_ID'].drop_duplicates()
print len(gold_umls)
print len(gold_drug)

376
446


In [213]:
gold_drug_indications = ind_umls_db3[ind_umls_db3['DrugBank_ID'].isin(gold_drug)] # SIDER의 indication 정보를 가지고 있는 gold drug
print len(gold_drug_indications)
print len(gold_drug_indications.drop_duplicates('DrugBank_ID')) # 446개중 431개가 SIDER에서 indication 정보를 주고 있음.
feature_indi = gold_drug_indications[~gold_drug_indications['UMLS_concept_id_for_MedDRA'].isin(gold_umls)] # 그중 association 정보가 아닌 애들
print len(feature_indi)
print len(feature_indi.drop_duplicates('UMLS_concept_id_for_MedDRA'))# indication 종류
print len(feature_indi.drop_duplicates('DrugBank_ID')) # feature로 쓰일 수 있는 indication 정보를 가진 drug 의 수

6693
431
5330
1346
412


In [215]:
feature_indi.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_gold_indication_except_asso.tsv", sep='\t', index=False)
feature_indi.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_gold_indication_except_asso.tsv", sep='\t', index=False)

In [233]:
# indication을 feature로 쓸 수 있는 drug-disease association
indi_drug = feature_indi['DrugBank_ID'].drop_duplicates()
a = gold[gold['DrugBank_ID'].isin(indi_drug)]
print len(a)
print len(a.drop_duplicates('DrugBank_ID'))
print len(a.drop_duplicates('OMIM_ID'))

1296
412
234


In [241]:
a.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_2_gold_asso_indi_ver.tsv", sep='\t', index=False)
a.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_2_gold_asso_indi_ver.tsv", sep='\t', index=False)

In [247]:
#indep의 indication에 맞춘것
feature_indi3 = feature_indi[feature_indi['UMLS_concept_id_for_MedDRA'].isin(feature_indi2['UMLS_concept_id_for_MedDRA'])]
indi_drug2 = feature_indi3['DrugBank_ID'].drop_duplicates()
aa = gold[gold['DrugBank_ID'].isin(indi_drug2)]
print len(aa)
print len(aa.drop_duplicates('DrugBank_ID'))
print len(aa.drop_duplicates('OMIM_ID'))

1253
390
232


In [ ]:
feature_indi2.drop_duplicates('UMLS_concept_id_for_MedDRA')

In [248]:
feature_indi3.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_3_gold_indication_indep_ver.tsv", sep='\t', index=False)
feature_indi3.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_3_gold_indication_indep_ver.tsv", sep='\t', index=False)
aa.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_4_gold_asso_indi_ver_indep_ver.tsv", sep='\t', index=False)
aa.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/3_4_gold_asso_indi_ver_indep_ver.tsv", sep='\t', index=False)

In [ ]:
# gold drug의 se와 indication이 얼마나 겹치는지 보자!

In [216]:
df_gold_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/4_gold_se_by_indep.tsv")
df_gold_se.head()

,DrugBank_ID,umls_id,event
0,DB00215,C0000737,Abdominal pain
1,DB00215,C0000786,Abortion spontaneous
2,DB00215,C0001824,Agranulocytosis
3,DB00215,C0002170,Alopecia
4,DB00215,C0002453,Amenorrhoea


In [176]:
gold_se = df_gold_se['umls_id'].drop_duplicates()
print len(feature_indi[feature_indi['UMLS_concept_id_for_MedDRA'].isin(gold_se)].drop_duplicates('UMLS_concept_id_for_MedDRA'))
print len(gold_se[gold_se.isin(feature_indi['UMLS_concept_id_for_MedDRA'])])

665
665


In [ ]:
#현재 665개가 겹친다. 어떤 drug는 특정 umls를 side effect로, 다른 drug는 umls를 indication으로 고려한다는 뜻.

In [ ]:
#independent의 sider

In [218]:
df_indep_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/3_indep_drug_se.tsv")

In [220]:
len(df_indep_se.drop_duplicates('DrugBank_ID'))

73

In [221]:
indep_drug = df_indep_se['DrugBank_ID'].drop_duplicates()
indep_drug_indications = ind_umls_db3[ind_umls_db3['DrugBank_ID'].isin(indep_drug)] 

In [223]:
print len(indep_drug_indications)
print len(indep_drug_indications.drop_duplicates('DrugBank_ID')) # 73개중 69가 SIDER에서 indication 정보를 주고 있음.
feature_indi2 = indep_drug_indications[~indep_drug_indications['UMLS_concept_id_for_MedDRA'].isin(gold_umls)] # 그중 association 정보가 아닌 애들
print len(feature_indi2)
print len(feature_indi2.drop_duplicates('UMLS_concept_id_for_MedDRA'))# indication 종류
print len(feature_indi2.drop_duplicates('DrugBank_ID')) # feature로 쓰일 수 있는 indication 정보를 가진 drug 의 수

1349
69
1095
557
68


In [231]:
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/3_indep_asso.tsv")
indi_drug2 = feature_indi2['DrugBank_ID'].drop_duplicates()
b = indep[indep['drug'].isin(indi_drug2)]
print len(b)
print len(b.drop_duplicates('drug'))
print len(b.drop_duplicates('disease'))

88
68
35


In [249]:
feature_indi2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/4_indep_indication.tsv", sep='\t', index=False)
feature_indi2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/4_indep_indication.tsv", sep='\t', index=False)
b.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/sider_indication/4_2_indep_asso_indi_ver.tsv", sep='\t', index=False)
b.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/sider_indication/4_2_indep_asso_indi_ver.tsv", sep='\t', index=False)